# Multi-pole Debye model for biological tissues

In the ``user_libs`` sub-package is a module called ``DebyeFit`` which can be used to to fit a multi-Debye expansion to dielectric data. As an adaptation of Debye model, the Cole-Cole model develops by introducing an exponent parameter $\alpha$. By applying the variable ```alpha```, the Cole-Cole model not only can achieve broader dispersions but also it flats the relaxation spectra. Due to these advantages and improvements, the **Cole-Cole model** is widely used in characterizing the complex permittivity of biological materials under high frequencies. Cole-Cole media cannot be directly implemented to FDTD, that's why approximations of the model (in the script by a multi-Debye function) are employed. In the gprMax code the **Cole-Cole model** could be approximated by using a variation of Havriliak-Negami function with ```beta = 1```, and given as

$$
\epsilon(\omega) = \epsilon_{\infty} + \frac{\Delta\epsilon}{1+\left(j\omega t_{0}\right)^{a}}
$$
where $\epsilon(\omega)$ is frequency dependent dielectric properties, $\Delta\epsilon$ - difference between the real permittivity at zero and infinity frequency.
$\tau_{0}$ is relaxation time,  $\epsilon_{\infty}$ - real part of relative permittivity at infinity frequency.

In the notebook to model biological tissues in wide range of frequences we used data from Gabriel et al. [1,2] and the ```HavriliakNegami``` class has the following structure:

```
HavriliakNegami(f_min, f_max,
                alpha, beta, e_inf, de, tau_0,
                sigma, mu, mu_sigma, material_name,
                number_of_debye_poles=-1, f_n=50,
                plot=False, save=True,
                optimizer=PSO_DLS,
                optimizer_options={})
```

* ``f_min`` is first bound of the frequency range used to approximate the given function (Hz),
* ``f_max`` is second bound of the frequency range used to approximate the given function (Hz),
* ``alpha`` is real positive float number which varies 0 < $\alpha$ < 1,
* ``beta`` is real positive float number which varies 0 < $\beta$ < 1,
* ``e_inf`` is a real part of relative permittivity at infinity frequency,
* ```de``` is a difference between the real permittivity at zero and infinity frequency,
* ```tau_0``` is a relaxation time,
* ```sigma``` is a conductivity (Siemens/metre),
* ```mu``` is a relative permeability,
* ```mu_sigma``` is a magnetic loss,
* ```material_name``` is definition of material name,
* ```number_of_debye_poles``` is choosen number of Debye poles,
* ```f_n``` is choosen number of frequences,
* ```plot``` is a switch to turn on the plotting,
* ```save``` is a switch to turn on the saving final material properties,
* ```optimizer``` is a choosen optimizer to fit model to dielectric data,
* ```optimizer_options``` is a dict for options of choosen optimizer.

[1] S Gabriel et al 1996 Phys. Med. Biol. 41 2271 <br>
[2] A Peyman and C Gabriel 2010 Phys. Med. Biol. 55 N413


In [6]:
import timeit
import os, sys
sys.path.append(os.path.abspath(os.path.join('..')))
from Debye_Fit import HavriliakNegami

tissues = { 'Bone': [2,  39.8, 8.42e-12, 0.72],
            'Fat': [2, 12.68,  10.49e-12, 0.85],
            'Skin': [3.0, 46.03, 7.80e-12, 0.66],
            'Tongue': [4.0, 53.00, 8.35e-12, 0.74],
            'Skull': [2.0, 42.42, 8.84e-12, 0.67]
          }
results = []
for key, value in tissues.items():
    print(f'Using {key}')

    start = timeit.default_timer()
    setup = HavriliakNegami(f_min=1e1, f_max=1e11,
                            alpha=value[3], beta=1,
                            e_inf=value[0], de=value[1], tau_0=value[2],
                            sigma=0, mu=0, mu_sigma=0,
                            material_name=key,
                            number_of_debye_poles=5, f_n=100,
                            plot=True, save=False,
                            optimizer_options={'seed':111})
    error, _ = setup.run()
    stop = timeit.default_timer()

    print(f'Time for {key} tissue in aproximating {setup.name}: {stop - start}')
    results.append({'tissue': key, 'duration': stop - start, 'average error': error})

print(f"{'Tissue':^70s}|{'Duration':^10s}|{'Average Error':^20s}")
for v in results:
    print(f"{v['tissue']:^70s}|{v['duration']:^10.4f}|{v['average error']:^20.4f}")
